In [58]:

import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate , train_test_split , StratifiedShuffleSplit
from sklearn import   linear_model, metrics
from sklearn.linear_model import SGDClassifier as SGD

In [59]:
train_directory = "data/train.csv"
train = pd.read_csv(train_directory, header=0 )
train['y'] = [1 if y == 'yes' else 0 for y in train['y']]

In [60]:
from sklearn.preprocessing import LabelEncoder
encoders = {}
for k,v in train.dtypes.iteritems():
    if v == 'object':
        encoder = LabelEncoder()
        train[k] = encoder.fit_transform(train[k])
        encoders[k] = encoder

In [350]:
cor = train.corr()['y'].drop(['y'], axis = 0)
cor.sort_values()
#посчитал корреляцию, выбрал фичи с наибольшей корреляцией(константа 0.1 отобралась перебором)

pdays            -0.344608
nr.employed      -0.332281
euribor3m        -0.278375
emp.var.rate     -0.259834
contact          -0.132529
campaign         -0.079928
default          -0.068977
cons.price.idx   -0.065774
loan             -0.011874
housing          -0.009219
day_of_week      -0.004273
month             0.007864
job               0.024329
cons.conf.idx     0.034768
marital           0.045862
age               0.053425
education         0.063527
poutcome          0.125979
previous          0.268134
duration          0.433596
Name: y, dtype: float64

In [351]:
#X = train[['age' , 'previous']][:]
features = cor[abs(cor) > 0.1].index.tolist()

X = train[features][:]
y = train['y'][:]

In [145]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, roc_auc_score
def get_cross_val_score(classifier, X, y):
    return cross_val_score(classifier, test_data, test_labels,
                         scoring=make_scorer(roc_auc_score), cv=5).mean() # я не понимаю, почему это отличается от scoring = 'roc_auc'

In [311]:
from sklearn.model_selection import GridSearchCV
param_grid = [
{'max_depth' : [None, 5, 10, 15, 20],
 'min_samples_split' : [4,5,6,7], 
 'min_samples_leaf' : [8,9,10,11,12,13],
}
]
RFC = RandomForestClassifier(max_features = None, n_estimators=30) # получены в результате предыдущих исследований
grid_search = GridSearchCV(RFC, param_grid, cv=5,
scoring=make_scorer(roc_auc_score))
grid_search.fit(X, y)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'max_depth': [None, 5, 10, 15, 20], 'min_samples_split': [4, 5, 6, 7], 'min_samples_leaf': [8, 9, 10, 11, 12, 13]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(roc_auc_score), verbose=0)

In [312]:
grid_search.best_params_

{'max_depth': 10, 'min_samples_leaf': 9, 'min_samples_split': 5}

In [353]:
best = grid_search.best_estimator_
get_cross_val_score(best, X, y)

0.8077074600217357

In [ ]:
'''
будь у меня время, я бы:
1) Изучил SGDClassifier более подробно, намёк из импортов я понял, а вот хорошего результата получить не смог
2) Провёл иной анализ фич, возможно бы сделал one-hot encoding для некоторых колонок, и использовал какие-нибудь другие способы анализа
3) Изучил случаные леса более подробно, вероятно я что-то не знаю
4) Использовал иные способы классификации(скормил бы всё нейронке, например)
'''